## Challenge:

We work at an importer and the price of our products is linked to a quote from:

* Dollar
* Euro
* Gold

We need to automatically get the price of these 3 items from the internet and know how much we should charge for our products, considering a contribution margin that we have in our database.

In [ ]:
pip install -U selenium

In [3]:
# Importando a biblioteca Selenium

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

# Abrindo uma página na web
navegador = webdriver.Chrome()

navegador.get("https://www.google.com/")



In [4]:
# Passo 1: Buscando a cotação do Dólar
navegador.find_element(By.XPATH,
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação dólar")

navegador.find_element(By.XPATH,
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

cotacao_dolar = navegador.find_element(By.XPATH,
    '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value") 
print(cotacao_dolar)

# Passo 2: Buscando a cotação do Euro
navegador.get("https://www.google.com/")
navegador.find_element(By.XPATH,
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação euro")
navegador.find_element(By.XPATH,
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

cotacao_euro = navegador.find_element(By.XPATH,
    '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")
print(cotacao_euro)

# Passo 3: Buscando a cotação do Ouro
navegador.get("https://www.melhorcambio.com/ouro-hoje")

cotacao_ouro = navegador.find_element(By.XPATH, '//*[@id="comercial"]').get_attribute("value")
cotacao_ouro = cotacao_ouro.replace(",", ".")
print(cotacao_ouro)

navegador.quit()

5.3822
5.5755
301.10


In [5]:
# Passo 4: Importando a lista de produtos
import pandas as pd

df = pd.read_excel("Produtos.xlsx")
display(df)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


In [13]:
# Passo 5: Recalcular o preço de cada produto

df.loc[df["Moeda"] == "Dólar", "Cotação"] = float(cotacao_dolar)
df.loc[df["Moeda"] == "Euro", "Cotação"] = float(cotacao_euro)
df.loc[df["Moeda"] == "Ouro", "Cotação"] = float(cotacao_ouro)

# Atualizar o preço de compra
df["Preço de Compra"] = df["Preço Original"] * df["Cotação"]

# Atualizar o preço de venda
df["Preço de Venda"] = df["Preço de Compra"] * df["Margem"]
df["Preço de Venda"] = df["Preço de Venda"].map("R${:.2f}".format)

display(df)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.3822,5382.146178,1.40,R$7535.00
1,Carro Renault,4500.00,Euro,5.5755,25089.750000,2.00,R$50179.50
2,Notebook Dell,899.99,Dólar,5.3822,4843.926178,1.70,R$8234.67
3,IPhone,799.00,Dólar,5.3822,4300.377800,1.70,R$7310.64
4,Carro Fiat,3000.00,Euro,5.5755,16726.500000,1.90,R$31780.35
5,Celular Xiaomi,480.48,Dólar,5.3822,2586.039456,2.00,R$5172.08
6,Joia 20g,20.00,Ouro,301.1000,6022.000000,1.15,R$6925.30


In [16]:
# Passo 6: Exportar o novos valores para uma planilha Excel
df.to_excel("Produtos Novo.xlsx", index=False)
